<a href="https://colab.research.google.com/github/Arseniy-Polyakov/russian_sign_language_recognition/blob/main/Sign_Language_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch --upgrade --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.2/287.2 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32

In [26]:
import re
import wandb
import pandas as pd
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    BartForConditionalGeneration,
    BartTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq
)
from datasets import load_dataset

In [15]:
wandb.init(
    project="marian-mt-translation",
    name="marian-mt-ru-rsl"
)

In [16]:
dataset = load_dataset(
    "csv",
    data_files={
        "train": "train.csv",
        "test": "test.csv"
    }
)
dataset

DatasetDict({
    train: Dataset({
        features: ['russian', 'rsl'],
        num_rows: 80
    })
    test: Dataset({
        features: ['russian', 'rsl'],
        num_rows: 20
    })
})

In [17]:
def tokenizing(tokenizer):
    def preprocess(batch):
        model_inputs = tokenizer(
            batch["russian"],
            truncation=True,
            padding="max_length",
            max_length=128
        )

        with tokenizer.as_target_tokenizer():
            labels = tokenizer(
                batch["rsl"],
                truncation=True,
                padding="max_length",
                max_length=128
            )
        labels_ids = labels["input_ids"]
        labels_ids = [
            [(tok if tok != tokenizer.pad_token_id else -100) for tok in sent]
            for sent in labels_ids
        ]

        model_inputs["labels"] = labels_ids
        return model_inputs
    return preprocess

In [18]:
# model_path = "facebook/bart-base"
# model_path = "google/mt5-small"
model_path = "hf-internal-testing/tiny-random-MarianMTModel"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# model = BartForConditionalGeneration.from_pretrained(model_path)
# tokenizer = BartTokenizer.from_pretrained(model_path)
# data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

tokenizer_config.json:   0%|          | 0.00/322 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


config.json:   0%|          | 0.00/948 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.01M [00:00<?, ?B/s]

In [19]:
preprocessing = tokenizing(tokenizer)
tokenized_dataset = dataset.map(preprocessing, batched=True, remove_columns=dataset["train"].column_names)

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [25]:
training_args = TrainingArguments(
    output_dir="marian-mt",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1000,
    learning_rate=5e-5,
    report_to="wandb",
    run_name="marian-mt-ru-rsl",
    eval_steps=500,
    metric_for_best_model="bleu",
    greater_is_better=True
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator
)

/tmp/ipython-input-2328599156.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


AttributeError: 'TrainingArguments' object has no attribute 'generation_config'

In [21]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


IndexError: index out of range in self

In [ ]:
wandb.finish()

In [ ]:
model.eval()

In [ ]:
def translate_texts(texts, model, tokenizer, device="cpu", max_length=128, num_beams=5):
    model.to(device)
    inputs = tokenizer(
        texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_beams=num_beams,
            early_stopping=True,
            no_repeat_ngram_size=3
        )

    translations = tokenizer.batch_decode(
        outputs,
        skip_special_tokens=True
    )
    return translations

In [ ]:
df = pd.read_csv("giga_chat_dataset.csv")
data_for_translation = df["russian"].to_list()

translations = translate_texts(
    data_for_translation,
    model,
    tokenizer
)
translations